In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

import pandas as pd
import string

def substrings_in_string(big_string, substrings):
  for substring in substrings:
    if str.find(str(big_string), str(substring)) != -1:
      return substring
  return np.nan

def preprocess(name):
  df = pd.read_csv(name)

  age_median = df['Age'].median()
  df['Age'].fillna(age_median, inplace=True)
  fare_median = df['Fare'].median()
  df['Fare'].fillna(fare_median, inplace=True)
  df = df.dropna(subset=['Embarked'])

  df = pd.get_dummies(df, columns=['Sex', 'Embarked'])
  
  df['Family_Size'] = df['SibSp']+df['Parch']
  df['Deck']=df['Cabin'].map(lambda x: substrings_in_string(x, ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']))
  
  df = pd.get_dummies(df, columns=['Deck'])

  return df

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np

df = preprocess('gdrive/My Drive/train.csv')
data = df.values

Y = data[:, 1].astype(int)
X = data[:, [2, 4, 8, 10, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22]]


df_test = preprocess('gdrive/My Drive/test.csv')
data_test = df_test.values

X_test = data_test[:, [1, 3, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21]]

In [0]:
from sklearn.ensemble import RandomForestClassifier

brain = RandomForestClassifier(n_estimators=10000, max_depth=4)
brain.fit(X, Y)

preds = brain.predict(X_test)
result = np.concatenate((np.array([data_test[:, 0]]).T, np.array([preds]).T), axis=1)

np.savetxt("gdrive/My Drive/result.csv", result, delimiter=",", fmt='%s')